In [1]:
# install missing packages
%pip install nltk
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 9.56 GB
Used RAM: 5.59 GB
Percentage Usage Of RAM: 38.2%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 13.21 GB
Used Disk: 11.78 GB
Percentage Usage Of Disk: 47.1%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from wrangle import prepare
from nnet_regressor import Regression

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
2023-10-29 18:45:57.724349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# get the data
sales = pd.read_csv("Amazon.csv")

In [5]:
# prepare the data for machine learning
sales = prepare(df=sales)

Data Wrangling:
> Removing Unnecessary Columns
> Transforming Product Name
> Transforming Category
> Transforming Actual Price
> Transforming Discount Percentage
> Transforming Rating
> Transforming Rating Count
> Aggregating By Product ID
> Shuffling The Data
0.12 Seconds


In [6]:
# get the testing data
y = sales[["actual_price"]]
X = sales.drop(columns="actual_price")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [ ]:
# build the model
print("\n---- Amazon Sales Regression Analysis ----\n")
model = Regression(
    name="Tensorflow Without Feature Engineering", 
    path=None,
    rename=False, 
    time=False, 
    text=True,
    binary=True, 
    imputation=True, 
    variance=True,
    scale=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    tune=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(sales)  # plot the dataset
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"> R2: {model.r2}")
    print(f"> RMSE: {model.rmse}")
    print(f"> In Control: {model.in_control}")


---- Amazon Sales Regression Analysis ----

Visualizing The Data:
> Plotting Correlations
> Plotting actual_price
> Plotting discount_percentage
> Plotting rating
> Plotting rating_count
> Plotting category
> actual_price vs. product_name
> actual_price vs. category
> discount_percentage vs. product_name
> discount_percentage vs. category
> rating vs. product_name
> rating vs. category
> rating_count vs. product_name
> rating_count vs. category
8.65 Seconds
Model Training:
> Transforming The Training Data
> Transforming Text Features
> Transforming Categorical Features
> Filling In Missing Values
> Removing Constant Features
> Scaling Features
> Training Neural Network
24.7 Minutes
Model Performance:
> Transforming The Testing Data
> Scoring The Model
24.88 Seconds
Model Indicators:
> Perturbing Features


In [ ]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

In [ ]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

In [ ]:
# save the machine learning pipeline
model.dump()

In [ ]:
# refit the model to include the test data
model.refit(testX, testy)